<a href="https://colab.research.google.com/github/apujais007/api-nuggets/blob/main/grade_fmt_detailed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import userdata
import os

# Set the OPENAI_API_KEY environment variable
os.environ["FMP_API_KEY"] = userdata.get('FMP')

print("FMP_API_KEY environment variable has been set.")

FMP_API_KEY environment variable has been set.


In [9]:
import requests
import pandas as pd

# Assuming fetch_sp500_symbols is defined in cell 2us4hx50Ph-0 or sH16SjyKWQNT
# Copying the function here for self-containment
def get_json(url, params=None):
    if params is None:
        params = {}
    params['apikey'] = os.environ["FMP_API_KEY"] # Use the API key directly
    r = requests.get(url, params=params)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"Error {r.status_code} for URL {url}")
        return None

def fetch_sp500_symbols(top_n=100):
    url = "https://financialmodelingprep.com/api/v3/sp500_constituent"
    data = get_json(url)
    if not data:
        return []
    return [item['symbol'] for item in data][:top_n]

# 1. Get the list of top 100 S&P 500 tickers using the existing function
top_100_tickers = fetch_sp500_symbols(top_n=500)

print(top_100_tickers)

['APP', 'EME', 'HOOD', 'IBKR', 'XYZ', 'TTD', 'DDOG', 'COIN', 'DASH', 'EXE', 'TKO', 'WSM', 'APO', 'LII', 'WDAY', 'TPL', 'DELL', 'ERIE', 'PLTR', 'SW', 'CRWD', 'GDDY', 'KKR', 'VST', 'GEV', 'SOLV', 'DECK', 'SMCI', 'BLDR', 'JBL', 'UBER', 'HUBB', 'LULU', 'VLTO', 'ABNB', 'BX', 'KVUE', 'PANW', 'AXON', 'FICO', 'BG', 'PODD', 'GEHC', 'STLD', 'FSLR', 'ACGL', 'TRGP', 'EQT', 'PCG', 'CSGP', 'INVH', 'KDP', 'ON', 'VICI', 'WBD', 'CPT', 'MOH', 'NDSN', 'CEG', 'FDS', 'EPAM', 'BRO', 'DAY', 'MTCH', 'TECH', 'MRNA', 'CRL', 'PTC', 'GNRC', 'NXPI', 'MPWR', 'TRMB', 'TSLA', 'POOL', 'TER', 'TDY', 'TYL', 'WST', 'DPZ', 'DXCM', 'CARR', 'OTIS', 'IR', 'PAYC', 'LYV', 'STE', 'ZBRA', 'ODFL', 'WRB', 'NOW', 'LVS', 'NVR', 'CDW', 'IEX', 'LDOS', 'TROW', 'TMUS', 'AMCR', 'CTVA', 'DD', 'DOW', 'FOX', 'FOXA', 'WAB', 'ATO', 'FANG', 'LW', 'JKHY', 'KEYS', 'FTNT', 'ROL', 'ANET', 'CPRT', 'CPAY', 'BR', 'EVRG', 'MSCI', 'TTWO', 'HII', 'NCLH', 'CDNS', 'SBAC', 'IQV', 'AOS', 'MGM', 'PKG', 'RMD', 'BKR', 'ALGN', 'EG', 'HLT', 'IT', 'AMD', 'ARE', '

In [12]:
import requests
from datetime import datetime, timedelta, date

def get_upgraded_downgraded_symbols(symbols, api_key, debug=False, test_date=None):
    base_url = "https://financialmodelingprep.com/stable/grades"

    today = datetime.today().date()

    if test_date:
        # Force valid_dates to include the test_date
        valid_dates = [datetime.strptime(test_date, "%Y-%m-%d").date()]
    else:
        # Default: last 3 trading days
        valid_dates = []
        day = today
        while len(valid_dates) < 3:
            if day.weekday() < 5:
                valid_dates.append(day)
            day -= timedelta(days=1)

    if debug:
        print("Valid dates being checked:", valid_dates)

    result = []

    for symbol in symbols:
        try:
            url = f"{base_url}?symbol={symbol}&apikey={api_key}"
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if not data:
                if debug:
                    print(f"{symbol}: no data returned")
                continue

            latest = data[0]
            grade_date = datetime.strptime(latest["date"], "%Y-%m-%d").date()
            action = latest["action"].lower()

            if debug:
                print(f"{symbol}: latest_date={grade_date}, action={action}")

            if grade_date in valid_dates and action in ("upgrade", "downgrade"):
                result.append(symbol)

        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    if debug:
        print("Final result:", result)

    return result

# Example usage:
symbols = ["DDOG", "AAPL", "MSFT", "TSLA"]
api_key = os.environ["FMP_API_KEY"]

#matches = get_upgraded_downgraded_symbols(symbols, api_key,debug=True)
#matches = get_upgraded_downgraded_symbols(top_100_tickers, api_key,debug=False, test_date="2025-10-01")
matches = get_upgraded_downgraded_symbols(top_100_tickers, api_key,debug=False)
print("Today's upgrades/downgrades:", matches)


Today's upgrades/downgrades: ['URI', 'FCX', 'FIS', 'T']


In [13]:
import requests
import pandas as pd

def get_top_grade_changes(symbols, api_key, top_n=3, debug=False):
    """
    Fetch the top N grade changes per symbol and return a combined DataFrame.

    Args:
        symbols (list): List of stock symbols.
        api_key (str): FMP API key.
        top_n (int): Number of top records to fetch per symbol.
        debug (bool): Print debug info if True.

    Returns:
        pd.DataFrame: Combined DataFrame of all symbols with top N grade changes.
    """
    base_url = "https://financialmodelingprep.com/stable/grades"
    all_records = []

    for symbol in symbols:
        try:
            url = f"{base_url}?symbol={symbol}&apikey={api_key}"
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if not data:
                if debug:
                    print(f"{symbol}: No data returned")
                continue

            # Take top N records
            for record in data[:top_n]:
                all_records.append(record)
                if debug:
                    print(record)

        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_records)
    return df

#symbols_to_check = ["DDOG", "AAPL", "MSFT"]  # Your symbols from upgrades/downgrades

symbols_to_check=matches
df_grades = get_top_grade_changes(symbols_to_check, api_key, top_n=3, debug=False)
pd.set_option('display.max_columns', None)

def highlight_action(row):
    if row['action'].lower() == 'upgrade':
        return ['background-color: lightgreen']*len(row)
    elif row['action'].lower() == 'downgrade':
        return ['background-color: salmon']*len(row)
    else:
        return ['']*len(row)

# Apply styling
styled_df = df_grades.style.apply(highlight_action, axis=1)
# Display in Jupyter/Colab
display(styled_df)


,symbol,date,gradingCompany,previousGrade,newGrade,action
0,URI,2025-10-01,Baird,Neutral,Outperform,upgrade
1,URI,2025-09-25,Keybanc,Overweight,Overweight,maintain
2,URI,2025-09-23,Citigroup,Buy,Buy,maintain
3,FCX,2025-09-30,B of A Securities,Neutral,Buy,upgrade
4,FCX,2025-09-25,UBS,Neutral,Neutral,maintain
5,FCX,2025-09-25,JP Morgan,Overweight,Overweight,maintain
6,FIS,2025-09-30,UBS,Neutral,Buy,upgrade
7,FIS,2025-08-06,UBS,Neutral,Neutral,maintain
8,FIS,2025-08-06,"Keefe, Bruyette & Woods",Outperform,Outperform,maintain
9,T,2025-10-01,Barclays,Overweight,Equal Weight,downgrade
